In [ ]:
#pip install ghostscript
#pip install camelot-py
#pip install cv2

In [22]:
import re

In [23]:
import camelot
import ghostscript
import numpy as np
import re

AAL_pdf = camelot.read_pdf('AAL-Asia-to-Australia-Schedule-23.05.2019.pdf', flavor = 'stream')
AAL_pdf_table = AAL_pdf[0]


In [24]:
#converts table to list
AAL_pdf_table_list = AAL_pdf_table.data
print(AAL_pdf_table_list)

[['', '', 'Asia to Australia Sailing Schedule\nAsia to Australia Sailing Schedule', '', '', '', ''], ['', '', '', '', '', '', '23/05/2019'], ['', '', '', 'AUSTRALIA EAST COAST SERVICE', '', '', ''], ['', 'AAL', 'MV', 'AAL', 'AAL', 'MV', 'AAL'], ['AUSTRALIA EAST COAST', '', '', '', '', '', ''], ['', 'Shanghai', 'Warnow Mars', 'Hong Kong', 'Shanghai', 'Warnow Mars', 'Hong Kong'], ['SERVICE', '', '', '', '', '', ''], ['', '904ES', '905ES', '906ES', '907ES', '908ES', '909ES'], ['JP', '', '', '', '', '', ''], ['Kobe**', '-', '-', '-', '30/06', '01/08', '01/09'], ['Yokohama**', '-', '-', '-', '-', '-', '-'], ['KR', '', '', '', '', '', ''], ['Pusan', '(04/04)', '(01/05)', '02/06', '03/07', '03/08', '03/09'], ['Incheon', '(08/04)', '(06/05)', '04/06', '05/07', '06/08', '06/09'], ['CN', '', '', '', '', '', ''], ['Bayuquan**', '-', '-', '12/06', '13/07', '15/08', '16/09'], ['Tianjin', '(13/04)', '(11/05)', '08/06', '09/07', '11/08', '11/09'], ['Shanghai', '(20/04)', '(22/05)', '16/06', '16/07', 

In [25]:
# length of rows and cols
AAL_rows = len(AAL_pdf_table_list)
AAL_cols = len(AAL_pdf_table_list[0])

In [26]:
def is_table_header(array):
    # pop 4 index
    arr = array.copy()
    value = arr.pop(3)
    all_empty = all("" == s or s.isspace() for s in arr)
    if (value != "" and all_empty) and (not value.isspace()):
        return True
    else: 
        return False
    
#use the next 5 list for the vessel

#remaining to be used for schedule until next table header found 
    #identify which one is country
def is_country(lists):
    copy_lists = lists.copy()
    value = copy_lists.pop(0)
    all_empty = all("" == s or s.isspace() for s in copy_lists)
    if (value != "" and all_empty) and (not value.isspace()):
        return True
    else: 
        return False
    
       

In [27]:
AAL_schedules = []
# print(AAL_pdf_table_list[1])
AAL_schedule_update = AAL_pdf_table_list[1][AAL_cols-1] 
# print(AAL_schedule_update)


no_of_table = 0
for i in range(0, AAL_rows): 
    AAL_list = AAL_pdf_table_list[i]
    if is_table_header(AAL_list):
        no_of_table = no_of_table + 1 
        AAL_schedules.append([])
    elif no_of_table == 0:
        continue           
    else: 
        AAL_schedules[no_of_table - 1].append(AAL_list)

# print(AAL_schedules)

['', '', '', '', '', '', '23/05/2019']
23/05/2019


In [28]:
AAL_vessel_schedules = []

for i in range(0, len(AAL_schedules)):
    AAL_schedule = AAL_schedules[i]
    #construct vessel name
    vessel_names = [m+" "+str(n) for m,n in zip(AAL_schedule[0], AAL_schedule[2])]   
    vessel_voyages = AAL_schedule[4]
    # construct headers
    for j in range(1, len(vessel_names)):
        vessel_name = vessel_names[j]
        vessel_voyage = vessel_voyages[j]
        current_country = None
        for k in range(5, len(AAL_schedule)):
            if is_country(AAL_schedule[k]):
                current_country = AAL_schedule[k][0]
            else:
                region = AAL_schedule[k][0]
                date = AAL_schedule[k][j]
                if date != '-':
                    AAL_vessel_schedules.append([vessel_name, vessel_voyage, current_country, region, date, AAL_schedule_update])
                    
# print(AAL_vessel_schedules)         

In [29]:

# AAL_singapore_voyage = [entry[1] for entry in AAL_vessel_schedules if re.search("[Singapore]", entry[3])]
AAL_singapore_voyage = [entry[1] for entry in AAL_vessel_schedules if entry[3] == "Singapore**"]
# print(AAL_singapore_voyage)

AAL_singapore_schedules = [entry for entry in AAL_vessel_schedules if entry[1] in AAL_singapore_voyage]

print(AAL_singapore_schedules)
print(len(AAL_singapore_schedules))

[['AAL Dampier', '04WS19', 'KR', 'Pusan', '07/06', '23/05/2019'], ['AAL Dampier', '04WS19', 'CN', 'Tianjin**', '05/06', '23/05/2019'], ['AAL Dampier', '04WS19', 'CN', 'Shanghai', '14/06', '23/05/2019'], ['AAL Dampier', '04WS19', 'TW', 'Kaohsiung', '22/06', '23/05/2019'], ['AAL Dampier', '04WS19', 'SG', 'Singapore**', '27/06', '23/05/2019'], ['AAL Dampier', '04WS19', 'AU', 'Port Hedland/Dampier**', '03/07', '23/05/2019'], ['AAL Dampier', '04WS19', 'AU', 'Fremantle', '05/07', '23/05/2019'], ['AAL Nanjing', '05WS19', 'KR', 'Pusan', '13/07', '23/05/2019'], ['AAL Nanjing', '05WS19', 'CN', 'Tianjin**', '11/07', '23/05/2019'], ['AAL Nanjing', '05WS19', 'CN', 'Shanghai', '16/07', '23/05/2019'], ['AAL Nanjing', '05WS19', 'TW', 'Kaohsiung', '21/07', '23/05/2019'], ['AAL Nanjing', '05WS19', 'SG', 'Singapore**', '26/07', '23/05/2019'], ['AAL Nanjing', '05WS19', 'AU', 'Port Hedland/Dampier**', '01/08', '23/05/2019'], ['AAL Nanjing', '05WS19', 'AU', 'Fremantle', '03/08', '23/05/2019'], ['AAL Dampier

In [9]:
def is_table_header(arr):
    # pop 4 index
    value = arr.pop(3)
    all_empty = all("" == s or s.isspace() for s in arr)
    if (value != "" and all_empty) and (not value.isspace()):
        return True
    else: 
        return False
    
header = ["", "", "", "Something", "", "", "", ""]
result = is_table_header(header)
if result == False:
    raise Exception("should be a header")

not_header = ["", "", "", "", "", "", ""]
result = is_table_header(not_header)
if result == True:
    raise Exception("should not be a header")

    
print("all tests pass.")

all tests pass.
